In [1]:
import pandas as pd
df = pd.read_csv("FSTs.txt", sep="\t")

# Compute LSBL for Kazakh population
df["LSBL_KAZ"] = (df["FST_EUR_KAZ"] + df["FST_EAS_KAZ"] - df["FST_EAS_EUR"]) / 2

# Save full results
df.to_csv("lsbl_results.csv", index=False)

# Function to compute empirical p-value (1 - percentile)
def empirical_pvalue(value, values_series):
    rank = (values_series > value).sum()
    total = len(values_series)
    return rank / total

# Example usage:
example_lsbl = df["LSBL_KAZ"].iloc[0]
pval = empirical_pvalue(example_lsbl, df["LSBL_KAZ"])
print(f"Empirical p-value for LSBL={example_lsbl:.4f}: {pval:.4f}")

Empirical p-value for LSBL=-0.0016: 0.2900


In [2]:
def lookup_snp_rank(snp_id, df, lsbl_column="LSBL_KAZ"):
    if snp_id not in df["rsID"].values:
        return f"SNP '{snp_id}' not found in the dataset."
    
    row = df[df["rsID"] == snp_id].iloc[0]
    lsbl_value = row[lsbl_column]
    
    # Rank is how many SNPs have higher LSBL (1-based)
    rank = (df[lsbl_column] > lsbl_value).sum() + 1
    total = len(df)
    pval = rank / total

    return {
        "SNP": snp_id,
        "LSBL": lsbl_value,
        "Rank": rank,
        "Total_SNPs": total,
        "Empirical_p_value": pval
    }

print(lookup_snp_rank("rs2108622", df))
print(lookup_snp_rank("rs3745274", df))
print(lookup_snp_rank("rs3745274", df))
print(lookup_snp_rank("rs4148323", df))
print(lookup_snp_rank("rs2070959", df))
print(lookup_snp_rank("rs4988235", df))
print(lookup_snp_rank("rs1573496", df))
print(lookup_snp_rank("rs671", df))
print(lookup_snp_rank("rs4148323", df))
print(lookup_snp_rank("rs2056900", df))
print(lookup_snp_rank("rs2076740", df))
print(lookup_snp_rank("rs189261858", df))

{'SNP': 'rs2108622', 'LSBL': np.float64(0.03196794), 'Rank': np.int64(3697), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.0074386916594902175)}
{'SNP': 'rs3745274', 'LSBL': np.float64(-0.0017728315), 'Rank': np.int64(147956), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.2977005851153732)}
{'SNP': 'rs3745274', 'LSBL': np.float64(-0.0017728315), 'Rank': np.int64(147956), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.2977005851153732)}
{'SNP': 'rs4148323', 'LSBL': np.float64(-0.0030528150000000004), 'Rank': np.int64(206545), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.41558684576938243)}
{'SNP': 'rs2070959', 'LSBL': np.float64(-0.009983164999999999), 'Rank': np.int64(348731), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.7016776794984265)}
{'SNP': 'rs4988235', 'LSBL': np.float64(-0.04315664999999999), 'Rank': np.int64(463761), 'Total_SNPs': 496996, 'Empirical_p_value': np.float64(0.9331282344324703)}
{'SNP': 'rs1573496', 'LSBL': np.flo

In [3]:
import pandas as pd

# List of SNPs to look up
snp_list = [
    "rs2108622", "rs3745274", "rs3745274", "rs4148323",
    "rs2070959", "rs4988235", "rs1573496", "rs671",
    "rs4148323", "rs2056900", "rs2076740", "rs189261858"
]

# Collect results
results = []
for snp in snp_list:
    res = lookup_snp_rank(snp, df)
    if isinstance(res, dict):
        results.append(res)
    else:
        print(res)  # Print warning if SNP not found

# Convert to DataFrame and display
if results:
    result_df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)
    print(result_df.to_string(index=False))


        SNP      LSBL   Rank  Total_SNPs  Empirical_p_value
  rs2108622  0.031968   3697      496996           0.007439
  rs3745274 -0.001773 147956      496996           0.297701
  rs4148323 -0.003053 206545      496996           0.415587
  rs2070959 -0.009983 348731      496996           0.701678
  rs4988235 -0.043157 463761      496996           0.933128
  rs1573496 -0.008546 334000      496996           0.672038
      rs671 -0.012194 365323      496996           0.735062
  rs2056900 -0.042473 462704      496996           0.931001
  rs2076740 -0.004245 241978      496996           0.486881
rs189261858  0.001528  74638      496996           0.150178


In [4]:
# Compute empirical p-values for LSBL_KAZ
df["Empirical_p_value"] = df["LSBL_KAZ"].rank(method="max", ascending=False) / len(df)
significant_rsids = df[df["Empirical_p_value"] < 0.05]["rsID"].tolist()
len(significant_rsids)

24849

In [5]:
# Step 1: Read SNPs to search for
with open("for_search_snps.txt") as f:
    search_snps = set(line.strip() for line in f if line.strip())

# Step 2: Convert significant_rsids to a set for fast lookup
significant_set = set(significant_rsids)

# Step 3: Find overlap
common_snps = search_snps & significant_set

# Step 4: Output
print(f"Found {len(common_snps)} SNPs from for_search_snps.txt in significant LSBL hits.")
print("Matching SNPs:", sorted(common_snps))


Found 7 SNPs from for_search_snps.txt in significant LSBL hits.
Matching SNPs: ['rs1057911', 'rs121909046', 'rs2108622', 'rs2297595', 'rs2302427', 'rs4665850', 'rs971074']


In [6]:
results = []
for snp in search_snps:
    res = lookup_snp_rank(snp, df)
    if isinstance(res, dict):
        results.append(res)
    else:
        print(res)  # Print warning if SNP not found

# Convert to DataFrame and display sorted by Empirical_p_value
if results:
    result_df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)
    result_df = result_df.sort_values("Empirical_p_value")
    print(result_df.to_string(index=False))

SNP 'rs181461079' not found in the dataset.
SNP 'rs28365085' not found in the dataset.
SNP 'rs80075254' not found in the dataset.
SNP 'rs2842949' not found in the dataset.
SNP 'rs1135840' not found in the dataset.
SNP 'rs111567390' not found in the dataset.
SNP 'rs145091069' not found in the dataset.
SNP 'rs193922516' not found in the dataset.
SNP 'rs1800111' not found in the dataset.
SNP 'rs28647808' not found in the dataset.
SNP 'rs61744232' not found in the dataset.
SNP 'rs1800120' not found in the dataset.
SNP 'rs138417770' not found in the dataset.
SNP 'rs35516286' not found in the dataset.
SNP 'rs2306282' not found in the dataset.
SNP 'rs1051519' not found in the dataset.
SNP 'rs114781869' not found in the dataset.
SNP 'rs1800098' not found in the dataset.
SNP 'rs55735703' not found in the dataset.
SNP 'rs12623642' not found in the dataset.
SNP 'rs34911792' not found in the dataset.
SNP 'rs3814400' not found in the dataset.
SNP 'rs183024280' not found in the dataset.
SNP 'rs20213

In [7]:
import pandas as pd

# Step 1: Read SNPs to search for
with open("snps_of_interest_list.txt") as f:
    search_snps = set(line.strip() for line in f if line.strip())

# Step 2: Convert significant_rsids to a set for fast lookup
significant_set = set(significant_rsids)

# Step 3: Find overlap
common_snps = search_snps & significant_set
print(f"Found {len(common_snps)} SNPs from snps_of_interest_list.txt in significant LSBL hits.")
#print("Matching SNPs:", sorted(common_snps))

# Step 4: Load MAFs and FSTs
def load_afreq(file, pop):
    df = pd.read_csv(file, sep="\t", header=0)
    
    # Fix column name for compatibility
    df.rename(columns={"#CHROM": "CHROM"}, inplace=True)

    # Select and rename relevant columns
    return df[["ID", "REF", "ALT", "ALT_FREQS"]].rename(columns={
        "REF": f"REF_{pop}",
        "ALT": f"ALT_{pop}",
        "ALT_FREQS": f"MAF_{pop}"
    })

def load_fst(file, name):
    df = pd.read_csv(file, sep="\t", header=None, names=["CHR", "POS", "ID", "N", f"FST_{name}"])
    return df[["ID", f"FST_{name}"]]

maf_eas = load_afreq("only_EAS.afreq", "EAS")
maf_eur = load_afreq("only_EUR.afreq", "EUR")
maf_kaz = load_afreq("only_KAZ.afreq", "KAZ")

fst_eas_kaz = load_fst("pairwise_fst.EAS.KAZ.fst.var", "EAS_KAZ")
fst_eas_eur = load_fst("pairwise_fst.EAS.EUR.fst.var", "EAS_EUR")
fst_eur_kaz = load_fst("pairwise_fst.EUR.KAZ.fst.var", "EUR_KAZ")

# Step 5: Build enriched result table
results = []
for snp in common_snps:
    res = lookup_snp_rank(snp, df)
    if isinstance(res, dict):
        results.append(res)
    else:
        print(res)

if results:
    result_df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)

    # Merge in MAFs and FSTs
    for table in [maf_eas, maf_eur, maf_kaz, fst_eas_kaz, fst_eas_eur, fst_eur_kaz]:
        result_df = result_df.merge(table, left_on="SNP", right_on="ID", how="left").drop(columns=["ID"])

    # Sort and display
    result_df = result_df.sort_values("Empirical_p_value")
    #print(result_df.to_string(index=False))

Found 156 SNPs from snps_of_interest_list.txt in significant LSBL hits.


/var/folders/d2/h4x4cs9x3gldcdn13dk70_xh0000gn/T/ipykernel_42398/2463019514.py:30: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", header=None, names=["CHR", "POS", "ID", "N", f"FST_{name}"])
/var/folders/d2/h4x4cs9x3gldcdn13dk70_xh0000gn/T/ipykernel_42398/2463019514.py:30: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", header=None, names=["CHR", "POS", "ID", "N", f"FST_{name}"])
/var/folders/d2/h4x4cs9x3gldcdn13dk70_xh0000gn/T/ipykernel_42398/2463019514.py:30: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", header=None, names=["CHR", "POS", "ID", "N", f"FST_{name}"])


In [8]:
# Keep only rows where all REF and all ALT alleles match across populations
consistent_df = result_df[
    (result_df["REF_EAS"] == result_df["REF_KAZ"]) &
    (result_df["REF_EAS"] == result_df["REF_EUR"]) &
    (result_df["ALT_EAS"] == result_df["ALT_KAZ"]) &
    (result_df["ALT_EAS"] == result_df["ALT_EUR"])
].reset_index(drop=True)

In [9]:
print(f"Filtered out {len(result_df) - len(consistent_df)} inconsistent SNPs.")
print(f"Remaining consistent SNPs: {len(consistent_df)}")

Filtered out 83 inconsistent SNPs.
Remaining consistent SNPs: 73


In [10]:
consistent_df.to_csv("consistent_lsbl_results.csv", index=False)
